<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 3 / NOTE 3*

# 📝 Assignment
---
# Feature Importances

### 1) 지속적으로 포트폴리오 프로젝트를 수행하세요
- 지속적으로 데이터를 랭글링 하고 추가적인 시각화 결과를 만드세요
- 작성한 코드마다 적절한 코멘트를 추가하세요
- 모델을 학습시키고 베이스라인 모델과 비교해 보세요
- **포트폴리오 프로젝트의 베이스라인 모델이 무엇인지 한 문장으로 설명하여 과제 제출폼에 제출하세요.** 

In [255]:
### 이곳에서 과제를 진행해 주세요 ### 

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [256]:
!pip install category_encoders

In [257]:
### (제출폼 과제) 이곳에서 과제를 진행해 주세요 ###
data = pd.read_csv('/content/drive/MyDrive/credit.csv')


 # 주제 : 외국인 노동자 신용평가 모델 만들기

 # 외국인 노동자 데이터만 추출
data = data.drop(data[data['foreign_worker'] == 'no'].index,axis=0)

data.default.value_counts()
# 채무 저위험군 == 1 
# 채무 고위험군 == 2

target = data['default']
train = data.drop(['default'],axis=1)

from category_encoders import OneHotEncoder
from category_encoders import OrdinalEncoder

ordinal = OrdinalEncoder(cols=['checking_balance','savings_balance','employment_length'])
train = ordinal.fit_transform(train)

onehot = OneHotEncoder()
train = onehot.fit_transform(train)
col = train.columns

In [258]:
train.columns

Index(['checking_balance', 'months_loan_duration', 'credit_history_1',
       'credit_history_2', 'credit_history_3', 'credit_history_4',
       'credit_history_5', 'purpose_1', 'purpose_2', 'purpose_3', 'purpose_4',
       'purpose_5', 'purpose_6', 'purpose_7', 'purpose_8', 'purpose_9',
       'purpose_10', 'amount', 'savings_balance', 'employment_length',
       'installment_rate', 'personal_status_1', 'personal_status_2',
       'personal_status_3', 'personal_status_4', 'other_debtors_1',
       'other_debtors_2', 'other_debtors_3', 'residence_history', 'property_1',
       'property_2', 'property_3', 'property_4', 'age', 'installment_plan_1',
       'installment_plan_2', 'installment_plan_3', 'housing_1', 'housing_2',
       'housing_3', 'existing_credits', 'dependents', 'telephone_1',
       'telephone_2', 'foreign_worker_1', 'job_1', 'job_2', 'job_3', 'job_4'],
      dtype='object')

In [259]:


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train = scaler.fit_transform(train)


In [260]:
train.shape

(963, 49)

In [261]:
#baseline 모델
data['default'].value_counts()

# 클래스 2개 중 1의 수가 확실히 많으므로 최빈값(1)을 기본 값으로 설정하여 베이스라인 성능을 측정할 수 있다고 생각했습니다.

1    667
2    296
Name: default, dtype: int64

In [262]:
# baseline 모델 성능 측정 
pred_y = [1] *len(train_y)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print("baseline roc_auc :",roc_auc_score(train_y,pred_y))
print("baseline f1 :",f1_score(train_y, pred_y))

baseline roc_auc : 0.5
baseline f1 : 0.6595477386934674


In [263]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train, target, train_size=0.8, random_state=24)

from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(max_depth = 20, n_estimators=100, random_state= 42)
rf.fit(train_X, train_y)
pred_y = rf.predict(val_X)
print("roc_auc :" ,roc_auc_score(val_y,pred_y))
print("f1 :",f1_score(val_y,pred_y))

roc_auc : 0.6916294088425236
f1 : 0.8501742160278745


In [264]:
# base line 보다 rou_auc점수는 0.19, f1은 0.2가량 크게 나왔으므로 성능이 크게 오른 것을 알 수 있다

### 2) Xgboost를 사용해 학습하고 기존 모델과 비교해 보세요.
- 본인 포트폴리오 데이터를 Xgboost로 학습하여 기존에 만들었던 모델과 비교합니다.
- **breast cancer 데이터를 Xgboost로 학습하고 f1 score를 제출하세요.(test 데이터에서 0.95이상)**

In [265]:
### (본인 포트폴리오 과제) 이곳에서 과제를 진행해 주세요 ### 

In [266]:
### (제출폼 과제) 이곳에서 과제를 진행해 주세요 ###

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer 

cancer_dataset = load_breast_cancer()

features = cancer_dataset.data
target = cancer_dataset.target
df = pd.DataFrame(data=features, columns=cancer_dataset.feature_names)
df['target'] = target

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=2)
print(X_train.shape, X_test.shape)

(455, 30) (114, 30)


In [267]:
!pip install category_encoders

In [268]:
from category_encoders import OrdinalEncoder

In [269]:
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    XGBClassifier(n_estimators=200
                  , random_state=2
                  , n_jobs=-1
                  , max_depth=7
                  , learning_rate=0.2))

In [270]:
from sklearn.metrics import f1_score
pipe.fit(X_train, y_train)
pred_y = pipe.predict(X_test)


f1_score(y_test, pred_y)

0.9635036496350365

In [289]:
# 내 포트폴리오 데이터 모델링

xgb = XGBClassifier(n_estimators=200
                  , random_state=2
                  , n_jobs=-1
                  , max_depth=7
                  , learning_rate=0.2)
xgb.fit(train_X,train_y)
pred = xgb.predict(val_X)
f1_score(val_y, pred)
# 랜덤포레스트와 비교하여 0.02정도 성능 오름

0.8745519713261649

### 3) Permutation importances를 계산하고 다른 중요도 결과와 비교해 보세요
- 본인 포트폴리오 데이터를 사용해 만든 모델의 Permutation importances를 계산하세요.
- **breast cancer 데이터를 사용해 만든 모델의 Permutation importances를 계산하고 가장 높은 값을 가지는 특성의 이름을 과제 제출폼에 제출하세요.**
    - eli5 PermutationImportance 사용
    - scoring = 'accuracy'
    - n_iter = 5
    - random_state = 2

In [271]:
### (본인 포트폴리오 과제) 이곳에서 과제를 진행해 주세요 ### 
!pip install eli5 

In [272]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('preprocessing', make_pipeline(OrdinalEncoder(), SimpleImputer())),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)) 
])

In [273]:
pipe.named_steps

{'preprocessing': Pipeline(memory=None,
          steps=[('ordinalencoder',
                  OrdinalEncoder(cols=None, drop_invariant=False,
                                 handle_missing='value', handle_unknown='value',
                                 mapping=None, return_df=True, verbose=0)),
                 ('simpleimputer',
                  SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                missing_values=nan, strategy='mean',
                                verbose=0))],
          verbose=False),
 'rf': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
         

In [274]:
pipe.fit(X_train,y_train)
pipe.score(X_test, y_test)

0.9473684210526315

In [275]:
### (제출폼 과제) 이곳에서 과제를 진행해 주세요 ###
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    pipe.named_steps['rf'],
    scoring = 'accuracy',
    n_iter = 5,
    random_state=2
)

X_test_transform = pipe.named_steps['preprocessing'].transform(X_test)
permuter.fit(X_test_transform,y_test)

PermutationImportance(cv='prefit',
                      estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fr

In [276]:
pd.Series(permuter.feature_importances_,cancer_dataset.feature_names).sort_values()

worst area                -4.440892e-17
mean radius                0.000000e+00
worst compactness          0.000000e+00
worst perimeter            0.000000e+00
fractal dimension error    0.000000e+00
symmetry error             0.000000e+00
concave points error       0.000000e+00
compactness error          0.000000e+00
perimeter error            0.000000e+00
texture error              0.000000e+00
smoothness error           0.000000e+00
mean fractal dimension     0.000000e+00
mean texture               0.000000e+00
mean perimeter             0.000000e+00
radius error               0.000000e+00
mean smoothness            0.000000e+00
mean compactness           0.000000e+00
worst fractal dimension    0.000000e+00
mean symmetry              0.000000e+00
mean area                  1.754386e-03
area error                 1.754386e-03
worst smoothness           1.754386e-03
worst radius               3.508772e-03
worst concavity            5.263158e-03
concavity error            5.263158e-03


In [277]:
eli5.show_weights(
    permuter, 
    top=1,
    feature_names=cancer_dataset.feature_names
)

In [278]:
# 내가 만든 모델의 주요 특성 찾기


pipe2 = Pipeline([
    ('preprocessing', make_pipeline(OrdinalEncoder(), SimpleImputer())),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)) 
])
pipe2.fit(train_X,train_y)
pipe2.score(val_X, val_y)

0.7927461139896373

In [279]:
permuter = PermutationImportance(
    pipe2.named_steps['rf'],
    scoring = 'accuracy',
    n_iter = 5,
    random_state=2
)

val_X_transform = pipe2.named_steps['preprocessing'].transform(val_X)
permuter.fit(val_X_transform,val_y)

PermutationImportance(cv='prefit',
                      estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fr

In [280]:
val_X.shape

(193, 49)

In [281]:
pd.Series(permuter.feature_importances_,col).sort_values()

property_4             -9.326425e-03
job_1                  -7.253886e-03
telephone_2            -7.253886e-03
housing_3              -6.217617e-03
installment_plan_2     -5.181347e-03
dependents             -5.181347e-03
job_3                  -4.145078e-03
personal_status_3      -4.145078e-03
property_1             -2.072539e-03
personal_status_4      -2.072539e-03
credit_history_2       -2.072539e-03
housing_1              -2.072539e-03
purpose_6              -1.036269e-03
other_debtors_3        -1.036269e-03
housing_2              -1.036269e-03
installment_plan_1     -1.036269e-03
property_2              0.000000e+00
foreign_worker_1        0.000000e+00
installment_plan_3      0.000000e+00
job_4                   0.000000e+00
credit_history_3        0.000000e+00
purpose_7               0.000000e+00
purpose_8               0.000000e+00
purpose_10              0.000000e+00
purpose_3               2.220446e-17
purpose_9               1.036269e-03
property_3              2.072539e-03
p

In [287]:
eli5.show_weights(
    permuter, 
    top=30,
    feature_names= col.tolist()
)

Weight,Feature
0.0705 ± 0.0181,checking_balance
0.0259 ± 0.0207,amount
0.0259 ± 0.0254,months_loan_duration
0.0187 ± 0.0192,employment_length
0.0135 ± 0.0106,residence_history
0.0104 ± 0.0217,age
0.0093 ± 0.0102,credit_history_5
0.0083 ± 0.0051,purpose_5
0.0083 ± 0.0083,personal_status_1
0.0073 ± 0.0051,credit_history_4


## 🔥 도전과제

### 4) Xgboost 외에 다른 부스팅 라이브러리로 모델을 학습시켜 Xgboost와 비교해 보세요.

In [282]:
### 이곳에서 과제를 진행해 주세요 ### 
from sklearn.ensemble import AdaBoostClassifier


abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
abc.fit(X_train,y_train)
pred_y = abc.predict(X_test)
f1_score(y_test,pred_y)

0.9558823529411764

In [283]:
# xgboost 성능 : 0.963503  adaboost 성능 : 0.955882 // xgboost의 성능이 조금 더 높게 측정됨